In speech processing we may use really huge networks cause majority of the processing bottleneck is on CPU and running the CTC function...

Also, CLARIN mobile corpus seems to be too small for advanced network

In [3]:
#!/usr/bin/python3

# For demonstration purposes - Paweł Tomasik

# for CLARIN_MOBILE - generally it is unnormalized

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from keras.models import Model
from keras.layers import LSTM, Conv1D, Dropout, LeakyReLU, Dense, Input, Lambda, TimeDistributed, Flatten, Conv2D, BatchNormalization, GRU, Bidirectional
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.initializers import Orthogonal

from keras.backend import ctc_batch_cost, expand_dims
import keras.backend as K

import editdistance  # For digit error rate
import keras
import librosa
import numpy as np
import os

import gc
X = Y = None
gc.collect()

LENGTH = 1600
RECS = 1384 * 3
TRANSL = 307

X = np.zeros([RECS, LENGTH, 20], np.float32)
Y = np.zeros([RECS, TRANSL], np.int16)
counter = 0

for i in range(3):
    Xpart = np.load("datasets/clarin-long/data/clarin-mfcc-rec-{}.npy".format(i))
    Ypart = np.load("datasets/clarin-long/data/clarin-mfcc-trans-{}.npy".format(i))
    recs = Xpart.shape[0]
    reclen = Xpart.shape[1]
    translen = Ypart.shape[1]
    X[counter : counter + recs, :reclen, :] = Xpart
    Y[counter : counter + recs, :translen] = Ypart
    counter += recs
    
print(counter, RECS)
counter //= 32
counter *= 32

X = X[:counter]
Y = Y[:counter]

#X = np.clip(X, -3, 3)
MEAN = X.mean()
STD = X.std()


4140 4152


In [ ]:
mc = keras.callbacks.ModelCheckpoint('models/mfcc-ctc-{epoch:08d}-bigger.h5', 
                                     save_weights_only=False, period=5)

from keras.constraints import max_norm

class MyCallback(keras.callbacks.Callback):
    def __init__(self, fname):
        self.fname = fname
        super(MyCallback, self).__init__()
        
    def on_epoch_end(self, epoch, logs={}):
        content = "Epoch: {}, loss: {};\n".format(epoch, logs.get('loss'))
        with open(self.fname, "a") as f:
            f.write(content)

logger = MyCallback("./training-bigger-log.txt")
NFEATS = 20

def mk_model(max_label_length):
    feature_input = Input(shape = (None, NFEATS))
    layer = feature_input
    layer = Lambda(lambda x: (x - MEAN) / STD)(layer)
    layer = Lambda(K.expand_dims)(layer)
    layer_1 = Conv2D(12, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer_1 = Conv2D(16, [5,1], activation='linear', strides=(2,1), kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    layer = TimeDistributed(Flatten())(layer)
    layer_1 = Conv1D(512, 5, activation='linear', kernel_initializer=Orthogonal(), padding='same')(layer)
    layer_2 = LeakyReLU(0.01)(layer_1)
    layer = BatchNormalization()(layer_2)
    for i in range(7):
        layer = Bidirectional(GRU(256, return_sequences = True, recurrent_dropout=0.01, kernel_initializer=Orthogonal(), activation='linear'))(layer)
        layer = LeakyReLU(0.01)(layer)
        layer = BatchNormalization()(layer)
    layer = Dense(1024)(layer)
    layer = LeakyReLU(0.01)(layer)
    layer = BatchNormalization()(layer)
    layer_15 = Dense(NPHONES + 1, activation = 'softmax')(layer)
    label_input = Input(shape = (max_label_length,))
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    loss_lambda = Lambda(lambda args:ctc_batch_cost(*args), output_shape=(1,), name='ctc')([label_input, layer_15, input_length, label_length])
    model = Model([feature_input, label_input, input_length, label_length], [loss_lambda])
    model.summary()
    predictive = Model(feature_input, layer_15)
    return model, predictive

def train(model, trainX, trainy, trainXl, trainyl, epochs = 10):
    # important: batch_size=1 bugs Tensorflow
    optimizer = Adam(0.0003, clipnorm=1.)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    return model.fit([trainX, trainy, trainXl, trainyl], np.zeros(trainX.shape[0]), epochs = epochs,
                     batch_size = 8, callbacks=[mc, logger])

def validate(predictions, valid_length, groundtruth, target_length):
    predictions = keras.backend.ctc_decode(predictions, valid_length, False, 1)
    predictions = predictions[0][0].eval(session=keras.backend.get_session())
    DERs = []
    for index in range(predictions.shape[0]):
        dist = float(editdistance.eval(
            [x for x in predictions[index, :] if x != -1],
            [x for x in groundtruth[index, :] if x != NPHONES]))
        DER = dist / target_length[index]
        DERs.append((DER, target_length[index]))
    return DERs

def try_else(exp, exp_else):
    try:
        return exp()
    except:
        return exp_else

if __name__=='__main__':
    data = X, Y
    NPHONES = Y.max()
    NFEATS = data[0].shape[2]
    X_lens = np.array([try_else(
            (lambda:np.where((x).mean(1) == (x).std(1))[0][0]),
            X.shape[1])
        for x in X])
    X_lens = np.ceil(X_lens / 4.0)
    Y_lens = np.array([np.where(x == NPHONES)[0] for x in data[1]])
    Y_lens = np.array([x[0] if len(x) else 0 for x in Y_lens])
    data = data[0][np.where(Y_lens)], data[1][np.where(Y_lens)], X_lens[np.where(Y_lens)].reshape(-1, 1), Y_lens[np.where(Y_lens)].reshape(-1, 1)
    print(list(map(lambda x:x.shape, data)))
    trn, predict = mk_model(data[1].shape[1])
    train(trn, *data, epochs=20) # at 300 it makes sensible predictions

[(4128, 1600, 20), (4128, 307), (4128, 1), (4128, 1)]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, 20)     0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, None, 20)     0           input_3[0][0]                    
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, None, 20, 1)  0           lambda_3[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, None, 20, 12) 72          lambda_4[0][0]                   
_______________________________________________________

Epoch 1/20
4128/4128 [==============================] - 2628s 637ms/step - loss: 322.1689
Epoch 2/20
4128/4128 [==============================] - 2613s 633ms/step - loss: 219.5594
Epoch 3/20
4128/4128 [==============================] - 2614s 633ms/step - loss: 187.0479
Epoch 4/20
4128/4128 [==============================] - 2614s 633ms/step - loss: 168.6160
Epoch 5/20
4128/4128 [==============================] - 2613s 633ms/step - loss: 155.5987
Epoch 6/20
1176/4128 [=======>......................] - ETA: 31:09 - loss: 145.7977

In [3]:
train(trn, *data, epochs=10)

Epoch 1/10
4128/4128 [==============================] - 2672s 647ms/step - loss: 144.0341
z:[0.99536806 0.03108552], w:[0.9999443 0.0070385]
Epoch 2/10
4128/4128 [==============================] - 2660s 644ms/step - loss: 133.9584
z:[0.9942052  0.02952564], w:[0.99994785 0.0070779 ]
Epoch 3/10
4128/4128 [==============================] - 2659s 644ms/step - loss: 121.4819
z:[0.99452925 0.02950952], w:[0.99997973 0.00634466]
Epoch 4/10
4128/4128 [==============================] - 2658s 644ms/step - loss: 120.9373
z:[0.99476683 0.02923215], w:[0.99995583 0.00588483]
Epoch 5/10
4128/4128 [==============================] - 2659s 644ms/step - loss: 150.5145
z:[0.9959114 0.0306349], w:[0.99997544 0.00698718]
Epoch 6/10
4128/4128 [==============================] - 2659s 644ms/step - loss: 128.0711
z:[0.9950818  0.02936261], w:[0.99998105 0.006141  ]
Epoch 7/10
4128/4128 [==============================] - 2659s 644ms/step - loss: 109.9221
z:[0.9949393  0.02898402], w:[0.9999628  0.00660334]
Epo

In [25]:
pr = predict.predict(X[:1])
pr = pr.argmax(2)[0]
pr

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [26]:
pr = predict.predict(X[:1])
pr = K.ctc_decode(pr, X_lens[:1])[0][0]
pr.eval(session=K.get_session())

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


array([[0]])

In [27]:
Y[:1]

array([[ 5,  3,  8, 32, 12, 36,  5, 32, 19,  8, 33, 31, 19, 23, 20,  5,
        19, 31, 32, 28,  7, 25, 16, 36, 13, 31,  2,  5, 31, 19,  5, 23,
        20, 19, 25,  5, 23, 17, 20,  5,  8,  5, 32, 32, 36, 32, 20,  5,
         2,  6,  3, 36, 17, 23,  5, 22, 36,  4, 24, 20,  5, 32, 12, 33,
        20, 12,  5,  8, 18,  5, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [28]:
import editdistance
editdistance.eval(list(pr.eval(session=K.get_session())[0]), list(Y[0, :Y_lens[0]])) / Y_lens[0]

1.0

In [29]:
pr = predict.predict(X[:160])
pr = K.ctc_decode(pr, X_lens[:160])[0][0]
hypos = [list(x) for x in pr.eval(session=K.get_session())]
lens = [x.index(-1) if -1 in x else len(x) for x in hypos]
hypos = [x[:lim] for x, lim in zip(hypos, lens)]
gts = [list(x) for x in Y[:160]]
lens = [x.index(37) if 37 in x else len(x) for x in gts]
gts = [x[:lim] for x, lim in zip(gts, lens)]
[editdistance.eval(gt, hypo) / len(gt) for gt, hypo in zip(gts, hypos)]

[1.0,
 0.9883720930232558,
 0.9838709677419355,
 1.0,
 0.9917355371900827,
 1.0,
 0.9814814814814815,
 0.9888888888888889,
 0.9852941176470589,
 0.991304347826087,
 0.9873417721518988,
 0.9861111111111112,
 0.9846153846153847,
 0.9852941176470589,
 0.9871794871794872,
 1.0,
 0.9868421052631579,
 1.0,
 0.9887640449438202,
 0.9879518072289156,
 1.0,
 0.9883720930232558,
 0.9896907216494846,
 1.0,
 0.9903846153846154,
 0.9900990099009901,
 0.987012987012987,
 0.9848484848484849,
 0.9861111111111112,
 0.98989898989899,
 0.9887640449438202,
 0.9861111111111112,
 0.9876543209876543,
 0.9850746268656716,
 0.9895833333333334,
 0.9871794871794872,
 0.9879518072289156,
 1.0,
 0.9883720930232558,
 0.9885057471264368,
 1.0,
 0.9880952380952381,
 0.9879518072289156,
 1.0,
 0.989010989010989,
 0.9782608695652174,
 0.9859154929577465,
 1.0,
 0.9864864864864865,
 0.9824561403508771,
 1.0,
 1.0,
 0.9907407407407407,
 0.989247311827957,
 0.9848484848484849,
 1.0,
 1.0,
 0.989010989010989,
 0.99074074074

In [ ]:
np.array(_).mean()

In [ ]:
predict.layers[1].z.eval(session=K.get_session())

In [ ]:
predict.layers[1].w.eval(session=K.get_session())

In [15]:
### Evaluation

import keras
from keras.constraints import max_norm
import keras.backend as K
import numpy as np
import tensorflow as tf
from keras.backend import ctc_batch_cost, expand_dims

MEAN = X.mean()
STD = X.std()

shift = np.exp(2 * np.pi * 1j / 512)

class CZT(keras.layers.Layer):
    def build(self, input_shape):
        self.z = self.add_weight(shape=[2], initializer=lambda shape: K.variable([np.real(shift), np.imag(shift)]),
                                 name='kernel_z', constraint=max_norm(1.00))
        self.w = self.add_weight(shape=[2], initializer=lambda shape: K.variable([1, 0]),
                                 name='kernel_w', constraint=max_norm(1.00))
    
    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = 257
        return tuple(output_shape)
    
    def call(self, inputs):
        ints = K.reshape(K.arange(257, dtype='complex64'), [257, 1])
        k = K.reshape(K.arange(512, dtype='complex64'), [1, 512])
        z = K.cast(self.z[0], dtype='complex64_ref') + 1j * K.cast(self.z[1], dtype='complex64_ref')
        w = K.cast(self.w[0], dtype='complex64_ref') + 1j * K.cast(self.w[1], dtype='complex64_ref')
        weights = K.dot(z * K.ones([257, 1], dtype='complex64'), K.reshape(w, [1, -1]) ** (-k)) ** (-ints)
        print(z.shape, w.shape, weights.shape)
        czt = K.dot(K.cast(inputs, dtype='complex64_ref'), K.transpose(weights))
        return K.abs(czt)

trainer = keras.models.load_model("models/mfcc-ctc-00000010-bigger.h5", {"CZT": CZT, "MEAN": MEAN, "STD": STD, "tf": tf, "ctc_batch_cost": ctc_batch_cost}, compile=False)

def try_else(exp, exp_else):
    try:
        return exp()
    except:
        return exp_else

predict = Model(trainer.layers[0].output, trainer.layers[-4].output)
data = X, Y
NPHONES = Y.max()
NFEATS = data[0].shape[2]
X_lens = np.array([try_else(
        (lambda:np.where((x).mean(1) == (x).std(1))[0][0]),
        X.shape[1])
    for x in X])
X_lens = np.ceil(X_lens / 4.0)
Y_lens = np.array([np.where(x == NPHONES)[0] for x in data[1]])
Y_lens = np.array([x[0] if len(x) else 0 for x in Y_lens])
data = data[0][np.where(Y_lens)], data[1][np.where(Y_lens)], X_lens[np.where(Y_lens)].reshape(-1, 1), Y_lens[np.where(Y_lens)].reshape(-1, 1)


() () (257, 512)
